In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
%matplotlib inline
from sklearn.linear_model import LinearRegression, LogisticRegression

In [2]:
os.listdir('./data')

['gender_submission.csv', 'test.csv', 'train.csv']

In [3]:
df_test = pd.read_csv('./data/test.csv')

In [4]:
df = df_test.set_index('PassengerId')

In [5]:
df[df.Fare > 200].sort_values('Fare')

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1216,1,"Kreuchen, Miss. Emilie",female,39.0,0,0,24160,211.3375,NaN,S
1299,1,"Widener, Mr. George Dunton",male,50.0,1,1,113503,211.5000,C80,C
1110,1,"Widener, Mrs. George Dunton (Eleanor Elkins)",female,50.0,1,1,113503,211.5000,C80,C
966,1,"Geiger, Miss. Amalie",female,35.0,0,0,113503,211.5000,C130,C
967,1,"Keeping, Mr. Edwin",male,32.5,0,0,113503,211.5000,C132,C
1048,1,"Bird, Miss. Ellen",female,29.0,0,0,PC 17483,221.7792,C97,S
973,1,"Straus, Mr. Isidor",male,67.0,1,0,PC 17483,221.7792,C55 C57,S
1006,1,"Straus, Mrs. Isidor (Rosalie Ida Blun)",female,63.0,1,0,PC 17483,221.7792,C55 C57,S
1094,1,"Astor, Col. John Jacob",male,47.0,1,0,PC 17757,227.5250,C62 C64,C


In [6]:
cond = df.Fare.isnull()
sub_value = df.Fare.mean()
df.Fare = np.where(cond, sub_value, df.Fare)

In [7]:
ticket_p = df.groupby('Ticket', as_index=False).agg({'Name': 'count'}).rename(columns={'Name':'t_count'}).sort_values('t_count', ascending=False)


,Ticket,t_count
319,PC 17608,5
13,113503,4
296,CA. 2343,4
181,347077,3
303,PC 17483,3
...,...,...
131,29105,1
129,28666,1
128,28664,1
127,28404,1


In [8]:
df_merge = df.merge(ticket_p, on='Ticket')
df_merge['Fare_per_person'] = df_merge.Fare/df_merge.t_count

In [9]:
cond = df_merge.Age.isnull()
sub_value = df_merge.Age.mean()
df_merge.Age = np.where(cond, sub_value, df_merge.Age)

In [10]:
cond = (df_merge.Sex == 'female')
sub_value = 1
df_merge.Sex = np.where(cond, sub_value, 0)

In [11]:
cond = df_merge.Embarked.isnull()
sub_value = 'S'
df_merge.Embarked = np.where(cond, 'S', df_merge.Embarked)

In [12]:
df_merge = df_merge.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df_merge.isnull().sum()

Pclass             0
Sex                0
Age                0
SibSp              0
Parch              0
Fare               0
Embarked           0
t_count            0
Fare_per_person    0
dtype: int64

In [13]:
ohe = OneHotEncoder()
X_cat = ohe.fit_transform(df_merge.Embarked.values.reshape(-1,1)).toarray()

In [14]:
X_short = df_merge.drop(['Embarked'], axis=1).values

In [17]:
X = np.hstack([X_cat, X_short])
X

array([[ 0.    ,  1.    ,  0.    , ...,  7.8292,  1.    ,  7.8292],
       [ 0.    ,  0.    ,  1.    , ...,  7.    ,  1.    ,  7.    ],
       [ 0.    ,  1.    ,  0.    , ...,  9.6875,  1.    ,  9.6875],
       ...,
       [ 0.    ,  0.    ,  1.    , ...,  7.25  ,  1.    ,  7.25  ],
       [ 0.    ,  0.    ,  1.    , ...,  8.05  ,  1.    ,  8.05  ],
       [ 1.    ,  0.    ,  0.    , ..., 22.3583,  1.    , 22.3583]])

In [19]:
X.shape

(418, 11)

In [20]:
def transform(df):
    df = df_test.set_index('PassengerId')
    cond = df.Fare.isnull()
    sub_value = df.Fare.mean()
    df.Fare = np.where(cond, sub_value, df.Fare)
    ticket_p = df.groupby('Ticket', as_index=False).agg({'Name': 'count'}).rename(columns={'Name':'t_count'}).sort_values('t_count', ascending=False)
    df_merge = df.merge(ticket_p, on='Ticket')
    df_merge['Fare_per_person'] = df_merge.Fare/df_merge.t_count
    cond = df_merge.Age.isnull()
    sub_value = df_merge.Age.mean()
    df_merge.Age = np.where(cond, sub_value, df_merge.Age)
    cond = (df_merge.Sex == 'female')
    sub_value = 1
    df_merge.Sex = np.where(cond, sub_value, 0)
    cond = df_merge.Embarked.isnull()
    sub_value = 'S'
    df_merge.Embarked = np.where(cond, 'S', df_merge.Embarked)
    df_merge = df_merge.drop(['Name', 'Ticket', 'Cabin'], axis=1)
    df_merge.isnull().sum()
    ohe = OneHotEncoder()
    X_cat = ohe.fit_transform(df_merge.Embarked.values.reshape(-1,1)).toarray()
    X_short = df_merge.drop(['Embarked'], axis=1).values
    X = np.hstack([X_cat, X_short])
    return X

In [22]:
X_test=transform(pd.read_csv('./data/test.csv'))

array([[ 0.    ,  1.    ,  0.    , ...,  7.8292,  1.    ,  7.8292],
       [ 0.    ,  0.    ,  1.    , ...,  7.    ,  1.    ,  7.    ],
       [ 0.    ,  1.    ,  0.    , ...,  9.6875,  1.    ,  9.6875],
       ...,
       [ 0.    ,  0.    ,  1.    , ...,  7.25  ,  1.    ,  7.25  ],
       [ 0.    ,  0.    ,  1.    , ...,  8.05  ,  1.    ,  8.05  ],
       [ 1.    ,  0.    ,  0.    , ..., 22.3583,  1.    , 22.3583]])